<a href="https://colab.research.google.com/github/Ananya29J/CodeHer_25-InSpec-AI/blob/main/Vehicle_Defect_Detection_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Mounting Drive***

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [43]:
import os

dataset_path = '/content/drive/MyDrive/Vehicle_Defect_Detection (1)/Vehicle_Dataset'
print("Contents of dataset folder:", os.listdir(dataset_path))


Contents of dataset folder: ['archive (1).zip', 'car_parts_prices.json', 'archive (2).zip']


***Unzip Dataset(T) File***

In [50]:
import zipfile

zip_path = '/content/drive/MyDrive/Vehicle_Defect_Detection (1)/Vehicle_Dataset/archive (2).zip'
extract_path = '/content/drive/MyDrive/Vehicle_Defect_Detection (1)/Vehicle_Dataset/'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("archive (2).zip extracted successfully!")


archive (2).zip extracted successfully!


***Converting Labelme to YOLO***

In [28]:
import os
import json

# Function to convert LabelMe annotations to YOLO format
def convert_labelme_to_yolo(annotation_file, image_width, image_height):
    with open(annotation_file) as f:
        data = json.load(f)

    yolo_annotations = []

    for shape in data['shapes']:
        label = shape['label']  # Get the label
        points = shape['points']  # Get the points of the polygon
        # Convert points to a bounding box (x_min, y_min, x_max, y_max)
        x_min = min([point[0] for point in points])
        y_min = min([point[1] for point in points])
        x_max = max([point[0] for point in points])
        y_max = max([point[1] for point in points])

        # Normalize the coordinates to be between 0 and 1
        center_x = (x_min + x_max) / 2 / image_width
        center_y = (y_min + y_max) / 2 / image_height
        width = (x_max - x_min) / image_width
        height = (y_max - y_min) / image_height


        class_id = 0

        # Add the annotation to the list
        yolo_annotations.append(f"{class_id} {center_x} {center_y} {width} {height}")

    return yolo_annotations

# Example for processing all annotations
annotations_path = '/content/drive/MyDrive/Vehicle_Defect_Detection (1)/Vehicle_Dataset/annotation LABELME'
image_folder = '/content/drive/MyDrive/Vehicle_Defect_Detection (1)/Vehicle_Dataset/images'

# Output folder for YOLO annotations
output_folder = '/content/drive/MyDrive/Vehicle_Defect_Detection (1)/Vehicle_Dataset/yolo_labels'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Loop through JSON annotation files and convert them
for annotation_file in os.listdir(annotations_path):
    if annotation_file.endswith('.json'):
        json_path = os.path.join(annotations_path, annotation_file)


        image_width, image_height = 800, 600

        # Convert the annotations
        yolo_annotations = convert_labelme_to_yolo(json_path, image_width, image_height)

        # Save to a text file in the YOLO format
        image_name = os.path.splitext(annotation_file)[0] + '.txt'  # Match with image name
        yolo_file_path = os.path.join(output_folder, image_name)

        with open(yolo_file_path, 'w') as f:
            f.write("\n".join(yolo_annotations))

        print(f"Converted {annotation_file} to YOLO format.")


Converted 1.json to YOLO format.
Converted 10.json to YOLO format.
Converted 14.json to YOLO format.
Converted 15.json to YOLO format.
Converted 16.json to YOLO format.
Converted 17.json to YOLO format.
Converted 18.json to YOLO format.
Converted 19.json to YOLO format.
Converted 2.json to YOLO format.
Converted 20.json to YOLO format.
Converted 21.json to YOLO format.
Converted 22.json to YOLO format.
Converted 23.json to YOLO format.
Converted 24.json to YOLO format.
Converted 25.json to YOLO format.
Converted 26.json to YOLO format.
Converted 27.json to YOLO format.
Converted 29.json to YOLO format.
Converted 3.json to YOLO format.
Converted 30.json to YOLO format.
Converted 31.json to YOLO format.
Converted 32.json to YOLO format.
Converted 33.json to YOLO format.
Converted 34.json to YOLO format.
Converted 36.json to YOLO format.
Converted 37.json to YOLO format.
Converted 38.json to YOLO format.
Converted 39.json to YOLO format.
Converted 4.json to YOLO format.
Converted 40.json 

In [13]:
import yaml

# Define class names based on the unique labels
class_names = [
    'broken - front headlight',
    'broken - windshield',
    'dent - door',
    'dent - front bumper',
    'dent - rear bumper',
    'scratch - door',
    'scratch - front bumper',
    'scratch - rear bumper'
]

# Create the dataset.yaml file
data = {
    'train': '/content/drive/MyDrive/Vehicle_Defect_Detection (1)/Vehicle_Dataset/images/train',
    'val': '/content/drive/MyDrive/Vehicle_Defect_Detection (1)/Vehicle_Dataset/images/val',
    'nc': len(class_names),
    'names': class_names
}

yaml_file_path = '/content/drive/MyDrive/custom_dataset.yaml'
with open(yaml_file_path, 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

print(f"custom_dataset.yaml created at {yaml_file_path}")


custom_dataset.yaml created at /content/drive/MyDrive/custom_dataset.yaml


In [18]:
# Clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5.git

# Change directory to yolov5 folder
%cd yolov5


Cloning into 'yolov5'...
remote: Enumerating objects: 17360, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 17360 (delta 36), reused 18 (delta 18), pack-reused 17308 (from 2)
Receiving objects: 100% (17360/17360), 16.25 MiB | 24.01 MiB/s, done.
Resolving deltas: 100% (11901/11901), done.
/content/yolov5/yolov5


In [24]:
%cd /content/yolov5
!python train.py --img 640 --batch 16 --epochs 10 --data /content/drive/MyDrive/custom_dataset.yaml --weights yolov5s.pt --cache

/content/yolov5
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-04-05 18:04:14.081609: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743876254.125052   33543 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743876254.138125   33543 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second tim

In [8]:
!pip install -U setuptools wheel
!pip install -U pip


In [7]:
!pip uninstall ultralytics -y
!pip install ultralytics


Found existing installation: ultralytics 8.3.102
Uninstalling ultralytics-8.3.102:
  Successfully uninstalled ultralytics-8.3.102
  Using cached ultralytics-8.3.102-py3-none-any.whl.metadata (37 kB)
Using cached ultralytics-8.3.102-py3-none-any.whl (993 kB)


In [10]:
import ultralytics
print(ultralytics.__version__)


8.3.102


***Unzip Dataset(S) File***

In [4]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/Vehicle_Defect_Detection (1)/Vehicle_Dataset/archive (1).zip'
extract_path = '/content/drive/MyDrive/Vehicle_Defect_Detection (1)/Vehicle_Dataset/'

# Extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Print the contents of the extracted folder
print("Contents of extracted folder:", os.listdir(extract_path))


Contents of extracted folder: ['archive (1).zip', 'car_parts_prices.json', 'archive (2).zip', 'annotation LABELME', 'data3a', 'yolo_labels']


***Model Definition (MNV2)***

In [2]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import os

# === 1. Set Paths ===
train_dir = "/content/drive/MyDrive/Vehicle_Defect_Detection (1)/Vehicle_Dataset/data3a/training"
val_dir = "/content/drive/MyDrive/Vehicle_Defect_Detection (1)/Vehicle_Dataset/data3a/validation"
img_size = (224, 224)
batch_size = 32

# === 2. Data Augmentation ===
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2]
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# === 3. Compute Class Weights ===
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights_dict = dict(enumerate(class_weights))

# === 4. Load Pretrained MobileNetV2 Base ===
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base model initially

# === 5. Add Custom Layers ===
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)                     # Increased dropout
x = Dense(128, activation='relu')(x)    # New layer
x = Dropout(0.3)(x)                     # New dropout
output = Dense(3, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# === 6. Compile Model ===
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# === 7. Callbacks ===
callbacks = [
    EarlyStopping(patience=3, restore_best_weights=True),
    ModelCheckpoint("best_model.keras", save_best_only=True, monitor='val_accuracy'),
    ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=2)
]

# === 8. Train Top Layers ===
initial_epochs = 15

steps_per_epoch = train_generator.samples // batch_size
validation_steps = val_generator.samples // batch_size

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=initial_epochs,
    class_weight=class_weights_dict,
    callbacks=callbacks,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)

# === 9. Fine-Tune Base Model ===
base_model.trainable = True
for layer in base_model.layers[:100]:
    layer.trainable = False  # Freeze first 100 layers

model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

fine_tune_epochs = 10

history_finetune = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=fine_tune_epochs,
    class_weight=class_weights_dict,
    callbacks=callbacks,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)

# === 10. Final Evaluation and Save ===
loss, accuracy = model.evaluate(val_generator)
print(f"Validation Accuracy: {accuracy*100:.2f}%")

# Save with accuracy in filename
model.save(f"fine_tuned_vehicle_model_acc_{accuracy*100:.2f}.keras")


Found 1383 images belonging to 3 classes.
Found 248 images belonging to 3 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
43/43 ━━━━━━━━━━━━━━━━━━━━ 331s 7s/step - accuracy: 0.3494 - loss: 1.3563 - val_accuracy: 0.5759 - val_loss: 0.9723 - learning_rate: 1.0000e-04
Epoch 2/15
 1/43 ━━━━━━━━━━━━━━━━━━━━ 54s 1s/step - accuracy: 0.2812 - loss: 1.2677

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


43/43 ━━━━━━━━━━━━━━━━━━━━ 14s 310ms/step - accuracy: 0.2812 - loss: 1.2677 - val_accuracy: 0.5580 - val_loss: 0.9707 - learning_rate: 1.0000e-04
Epoch 3/15
43/43 ━━━━━━━━━━━━━━━━━━━━ 202s 4s/step - accuracy: 0.4126 - loss: 1.1194 - val_accuracy: 0.6205 - val_loss: 0.8975 - learning_rate: 1.0000e-04
Epoch 4/15
43/43 ━━━━━━━━━━━━━━━━━━━━ 20s 441ms/step - accuracy: 0.3750 - loss: 1.2400 - val_accuracy: 0.5938 - val_loss: 0.9036 - learning_rate: 1.0000e-04
Epoch 5/15
43/43 ━━━━━━━━━━━━━━━━━━━━ 143s 3s/step - accuracy: 0.4798 - loss: 1.0431 - val_accuracy: 0.6295 - val_loss: 0.8758 - learning_rate: 1.0000e-04
Epoch 6/15
43/43 ━━━━━━━━━━━━━━━━━━━━ 13s 290ms/step - accuracy: 0.4062 - loss: 1.1241 - val_accuracy: 0.6518 - val_loss: 0.8634 - learning_rate: 1.0000e-04
Epoch 7/15
43/43 ━━━━━━━━━━━━━━━━━━━━ 142s 3s/step - accuracy: 0.5071 - loss: 1.0047 - val_accuracy: 0.6741 - val_loss: 0.8434 - learning_rate: 1.0000e-04
Epoch 8/15
43/43 ━━━━━━━━━━━━━━━━━━━━ 19s 421ms/step - accuracy: 0.4375 - l

***Evaluate and Save***

In [7]:
# Evaluate model performance
loss, accuracy = model.evaluate(val_generator)
print(f"Validation Accuracy: {accuracy*100:.2f}%")

# Create folder path
save_path = "/content/drive/My Drive/Vehicle_Defect_Detection/models/"
os.makedirs(save_path, exist_ok=True)

# Save the model
model.save(save_path + "vehicle_damage_model.h5")

8/8 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.6987 - loss: 0.7367


Validation Accuracy: 68.95%


***Load Model***

In [55]:
from tensorflow.keras.models import load_model

# Load previously trained model
model = load_model("vehicle_damage_model.h5")


***Image Augmentation + Normalization***

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Correct dataset paths
train_dir = "/content/vehicle_data/data3a/training"
val_dir = "/content/vehicle_data/data3a/validation"

# Image augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Load images again to refresh class count
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 1383 images belonging to 3 classes.
Found 248 images belonging to 3 classes.


***Mounting Test Images***

In [16]:
import os

# Define the path to test images
test_images_dir = '/content/drive/My Drive/Colab_Test_Images'

# List all files in the test images directory
test_images = os.listdir(test_images_dir)

# Display the list of test images
print(test_images)


['Severe.jpg', 'Moderate.jpeg', 'Minor.jpg']


***Car Parts Price File Load***

In [4]:
import json

# Define path
json_file_path = "/content/drive/My Drive/Vehicle Dataset/car_parts_prices.json"

# Load JSON data
with open(json_file_path, 'r') as file:
    car_parts_prices = json.load(file)

# View a sample
print(list(car_parts_prices.items())[:5])  # See first 5 entries


[('HONDA', {'City': {'Bonnet': 15000, 'Bumper': 10000, 'Dickey': 8000, 'Door': 20000, 'Fender': 5000, 'Light': 3000, 'Windshield': 8000}, 'Amaze': {'Bonnet': 12000, 'Bumper': 8000, 'Dickey': 6000, 'Door': 18000, 'Fender': 4000, 'Light': 2500, 'Windshield': 7000}, 'WR-V': {'Bonnet': 16000, 'Bumper': 11000, 'Dickey': 9000, 'Door': 22000, 'Fender': 6000, 'Light': 3500, 'Windshield': 9000}, 'Jazz': {'Bonnet': 14000, 'Bumper': 9000, 'Dickey': 7000, 'Door': 19000, 'Fender': 4500, 'Light': 2800, 'Windshield': 8000}, 'HR-V': {'Bonnet': 18000, 'Bumper': 12000, 'Dickey': 10000, 'Door': 24000, 'Fender': 7000, 'Light': 4000, 'Windshield': 10000}, 'Pilot': {'Bonnet': 22000, 'Bumper': 15000, 'Dickey': 13000, 'Door': 28000, 'Fender': 8000, 'Light': 5000, 'Windshield': 12000}, 'CR-V': {'Bonnet': 20000, 'Bumper': 13000, 'Dickey': 11000, 'Door': 26000, 'Fender': 7500, 'Light': 4500, 'Windshield': 11000}, 'Accord': {'Bonnet': 22000, 'Bumper': 15000, 'Dickey': 13000, 'Door': 28000, 'Fender': 8000, 'Light'

***Cloning into GitHub***

In [29]:
!git clone https://github.com/Ananya29J/CodeHer_25-InSpec-AI.git


Cloning into 'CodeHer_25-InSpec-AI'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (6/6), done.


In [32]:
!cp /content/drive/My\ Drive/Vehicle_Defect_Detection/models/vehicle_damage_model.h5 /content/CodeHer_25-InSpec-AI/
